In [1]:
import pandas as pd
import csv
from scipy.spatial import distance
import numpy as np
from sklearn.preprocessing import MinMaxScaler

In [2]:
RB_Combine = pd.read_csv('D:/Python/NFL Combine/RB_Combine.csv')
RB_Combine =RB_Combine.drop('Unnamed: 0',1)

# All names of RBs are unique
len(RB_Combine['Name'].unique())

# Columns in DF
RB_Combine.columns

X = pd.DataFrame(RB_Combine['Name'])
Y = RB_Combine[['Height', 'Weight', 'Forty_Yard', 'Bench_Press', 'Vert_Leap', 'Broad_Jump', 'Shuttle']]

# Scale Data

Scaler = MinMaxScaler(feature_range=(0,1))
Y_Scaled = Scaler.fit_transform(Y)

# Y.to_csv('D:/Python/NFL Combine/Y_RB.csv')
# Xa = X.values
Ya = Y_Scaled
Xa = X.values

In [3]:
def dist(x,y):
   return np.sqrt(np.sum((x-y)**2))

dist(Ya[1],Ya[2])

def takeSecond(elem):
    return elem[1]

In [4]:
def NN(person):

    Temp = []

    for itm2 in range(0,len(Ya)):
        distance = dist(Ya[[X[X.Name == person].index[0]]], Ya[[itm2]])
        Temp.append([itm2,distance])
        Temp.sort(key=takeSecond)

    Temp = Temp[0:6]

    df0 = pd.DataFrame(RB_Combine.iloc[Temp[0][0]])
    df1 = pd.DataFrame(RB_Combine.iloc[Temp[1][0]])
    df2 = pd.DataFrame(RB_Combine.iloc[Temp[2][0]])
    df3 = pd.DataFrame(RB_Combine.iloc[Temp[3][0]])
    df4 = pd.DataFrame(RB_Combine.iloc[Temp[4][0]])
    df5 = pd.DataFrame(RB_Combine.iloc[Temp[5][0]])
    
    df0 = df0.T
    df1 = df1.T
    df2 = df2.T
    df3 = df3.T
    df4 = df4.T
    df5 = df5.T
    
    frames = [df0, df1, df2, df3, df4, df5]
    
    result = pd.concat(frames)
    
    Distances_List = [Temp[0][1],Temp[1][1],Temp[2][1],Temp[3][1],Temp[4][1],Temp[5][1]]
    
    result['Eucl. Distance'] = Distances_List
    
    result['Eucl. Distance'] = result['Eucl. Distance'].round(3)
    
    result['Rank'] = ['-',1,2,3,4,5]
    
    result = result[['Rank','Year', 'Name', 'College', 'Pos', 'Height', 'Weight', 'Forty_Yard',
       'Bench_Press', 'Vert_Leap', 'Broad_Jump', 'Shuttle', 'Eucl. Distance']]
    
    return(result)

In [63]:
NN('''Jay Hinton''')

,Rank,Year,Name,College,Pos,Height,Weight,Forty_Yard,Bench_Press,Vert_Leap,Broad_Jump,Shuttle,Eucl. Distance
237,-,1999,Jay Hinton,Morgan State (MD),RB,71,200,4.29,18,43.5,134,3.86,0.000
254,1,2000,Curtis Keaton,James Madison (VA),RB,70,214,4.41,23,41.5,131,3.93,0.329
229,2,1999,Jerry Azumah,New Hampshire,RB,70,195,4.5,15,43.5,129,3.86,0.357
63,3,1990,Latin Berry,Oregon,RB,69.6,196,4.45,20,40,131,4.02,0.377
152,4,1993,Kevin Williams,UCLA,RB,72.5,212,4.44,17,41.5,137,4.11,0.394
149,5,1993,Robert Smith,Ohio State,RB,72.3,195,4.47,16,39,134,4.03,0.420


# Athletic Score

In [68]:
AthleticDF = pd.DataFrame(Y_Scaled)

AthleticDF = AthleticDF.drop(0,1)
AthleticDF = AthleticDF.drop(1,1)
AthleticDF.columns = ['Forty_Yard','Bench_Press', 'Vert_Leap','Broad_Jump','Shuttle']
AthleticDF

RB_Names = pd.DataFrame(RB_Combine[['Year','Name']])
AthleticDF = pd.merge(RB_Names,AthleticDF, how='left', left_index=True, right_index=True)
AthleticDF = AthleticDF[(AthleticDF.Year == 2018)|(AthleticDF.Year == 2017)|(AthleticDF.Year == 2016)]
# AthleticDF.sort_values('Shuttle', ascending=False)

In [69]:
# Weighting Modifications

AthleticDF['Forty_Yard'] = 1-AthleticDF['Forty_Yard']
AthleticDF['Shuttle'] = 1-AthleticDF['Shuttle']

In [71]:
Speed_Weight = 0.3
Explosive_Weight = 0.175
Bench_Weight = 0.05


AthleticDF['Athletic Score'] = Speed_Weight*AthleticDF['Forty_Yard'] + Bench_Weight*AthleticDF['Bench_Press'] + Explosive_Weight*AthleticDF['Vert_Leap'] + Explosive_Weight*AthleticDF['Broad_Jump'] + Speed_Weight*AthleticDF['Shuttle']
AthleticDF.sort_values('Athletic Score', ascending=False)

,Year,Name,Forty_Yard,Bench_Press,Vert_Leap,Broad_Jump,Shuttle,Athletic Score
479,2016,Daniel Lasco,0.757143,0.700000,0.888889,0.950,0.584906,0.759420
500,2018,Nick Chubb,0.671429,0.900000,0.722222,0.775,0.594340,0.686744
504,2018,Justin Jackson,0.671429,0.366667,0.722222,0.625,0.764151,0.684771
482,2016,Dan Vitale,0.557143,0.933333,0.722222,0.650,0.716981,0.669043
506,2018,Bo Scarbrough,0.671429,0.400000,0.805556,0.800,0.509434,0.655231
489,2017,Aaron Jones,0.614286,0.466667,0.666667,0.750,0.641509,0.647989
493,2017,Christian McCaffrey,0.728571,0.266667,0.666667,0.600,0.622642,0.640364
492,2017,Marlon Mack,0.700000,0.433333,0.555556,0.700,0.622642,0.638181
501,2018,Chase Edmonds,0.628571,0.566667,0.472222,0.625,0.764151,0.638164
475,2016,Kenyan Drake,0.771429,0.266667,0.500000,0.650,0.632075,0.635635
